jkzjksj

In [ ]:
# Define data paths
TRAIN_DIR = "../data/train"
TEST_DIR = "../data/test"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Explore dataset structure
def explore_dataset(data_dir):
    classes = ['MALNUTRITION', 'NUTRITION']
    data_info = {}
    
    for class_name in classes:
        class_path = os.path.join(data_dir, class_name)
        if os.path.exists(class_path):
            image_count = len([
                f for f in os.listdir(class_path)
                if f.lower().endswith(('.jpeg', '.jpg', '.png'))
            ])
            data_info[class_name] = image_count
        else:
            data_info[class_name] = 0
    
    return data_info

# Explore training and test data

train_info = explore_dataset(TRAIN_DIR)
test_info = explore_dataset(TEST_DIR)

for class_name, count in train_info.items():
    print(f"  {class_name}: {count} images")

print(f"\nTest Data:")
for class_name, count in test_info.items():
    print(f"  {class_name}: {count} images")

print(f"\nTotal Training Images: {sum(train_info.values())}")
print(f"Total Test Images: {sum(test_info.values())}")

# Calculate class distribution
total_train = sum(train_info.values())
if total_train > 0:
    print(f"\nClass Distribution (Training):")
    for class_name, count in train_info.items():
        percentage = (count / total_train) * 100
        print(f"  {class_name}: {percentage:.1f}%")

 Dataset Exploration:
  MALNUTRITION: 0 images
  NUTRITION: 0 images

Test Data:
  MALNUTRITION: 1004 images
  NUTRITION: 1005 images

Total Training Images: 0
Total Test Images: 2009


In [ ]:
#  Model Creation
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Load base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Build classification model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(train_gen.num_classes, activation='softmax')
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(train_gen, validation_data=val_gen, epochs=10)

# Save model
model.save('../models/mobilenetv2_model.h5')


#  Model Testing

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Predict on test data
preds = model.predict(test_gen)
y_pred = np.argmax(preds, axis=1)
y_true = test_gen.classes
class_labels = list(test_gen.class_indices.keys())

# Classification report
print(classification_report(y_true, y_pred, target_names=class_labels))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_labels, yticklabels=class_labels, cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


# Model Retraining

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

def retrain_model(new_data_dir):
    # Load new data
    new_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    new_train_gen = new_datagen.flow_from_directory(
        new_data_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )
    new_val_gen = new_datagen.flow_from_directory(
        new_data_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )

    # Load existing model
    model = load_model('../models/mobilenetv2_model.h5')

    # Retrain
    model.fit(new_train_gen, validation_data=new_val_gen, epochs=5)

    # Save updated model
    model.save('../models/mobilenetv2_model_retrained.h5')
    print("Model retrained and saved.")

# Example usage:
# retrain_model('../data/train/new_uploads')
